

---

## Problem: 3262. Find Overlapping Shifts

### Description

You are given a table **EmployeeShifts** with the following schema:

```
EmployeeShifts
• employee_id   | int
• start_time    | time
• end_time      | time
```

Each row represents a shift worked by an employee, with a start time and an end time.

You need to return a result set (table) with the following columns:

* `employee_id`
* `overlapping_shifts` — the count of overlapping shifts for that employee

But only include employees who have **at least one** overlapping shift. The output should be sorted by `employee_id` in ascending order.

Two shifts **overlap** if:

* They belong to the same employee.
* One shift’s start time is strictly before the other’s start time.
* The earlier shift’s end time is strictly greater than the later shift’s start time.

---

### Example

Input (EmployeeShifts):

| employee_id | start_time | end_time |
| ----------- | ---------- | -------- |
| 1           | 08:00:00   | 12:00:00 |
| 1           | 11:00:00   | 15:00:00 |
| 1           | 14:00:00   | 18:00:00 |
| 2           | 09:00:00   | 17:00:00 |
| 2           | 16:00:00   | 20:00:00 |
| 3           | 10:00:00   | 12:00:00 |
| 3           | 13:00:00   | 15:00:00 |
| 3           | 16:00:00   | 18:00:00 |
| 4           | 08:00:00   | 10:00:00 |
| 4           | 09:00:00   | 11:00:00 |

Output:

| employee_id | overlapping_shifts |
| ----------- | ------------------ |
| 1           | 2                  |
| 2           | 1                  |
| 4           | 1                  |

**Explanation:**

* **Employee 1** has three shifts:

  * 08:00–12:00
  * 11:00–15:00
  * 14:00–18:00
    The first overlaps with the second, and the second overlaps with the third, so there are **2** overlapping shifts.

* **Employee 2** has two shifts:

  * 09:00–17:00
  * 16:00–20:00
    They overlap with each other, so **1** overlapping shift.

* **Employee 3** has three shifts:

  * 10:00–12:00
  * 13:00–15:00
  * 16:00–18:00
    None overlap, so employee 3 is **not** included in the output.

* **Employee 4** has two shifts:

  * 08:00–10:00
  * 09:00–11:00
    They overlap, so **1** overlapping shift.

---


### Solution 1

In [ ]:
WITH CTE AS
(
    SELECT *, LEAD(start_time) OVER(PARTITION BY EMPLOYEE_ID ORDER BY NULL) as NEXT_START_TIME
    FROM EMPLOYEESHIFTS
),
WITH CTE2 AS
(
    SELECT *, CASE WHEN end_time > NEXT_START_TIME THEN AND NEXT_START_TIME is NOT NULL THEN 1 ELSE 0 END AS OVERLAPPING
)

SELECT employee_id, sum(OVERLAPPING) as OVERLAPPING_SHIFTS
FROM CTE2
GROUP BY employee_id
ORDER BY employee_id


 WRONG BECAUSE an EMPLIOYEE can have multiple shifts which are not sequential. So using LEAD() will not count all the shifts!

### Solution 2

In [ ]:
WITH CTE AS
(
    SELECT a.employee_id, a.start_time, a.end_time, b.start_time
    from EMPLOYEESHIFTS a
    JOIN EMPLOYEESHIFTS b
    on a.employee_id = b.employee_id
    AND a.start_time > b.start_time
    AND a.start_time < b.end_time
)

select employee_id, count(*) as OVERLAPPING_SHIFTS
FROM CTE
GROUP BY employee_id
ORDER BY employee_id

### Efficient Approach

Nice — here’s a **correct, efficient** approach you can use. It sorts each employee’s shifts and then does a single linear scan (via a window aggregate) to detect overlaps with *any previous* shift. This avoids the quadratic self-join while still catching overlaps that aren’t only between adjacent rows.

---

# Idea (short)

For every shift of an employee, compute the maximum `end_time` among **all previous shifts** (ordered by `start_time`). If the current shift's `start_time` is strictly less than that previous-max `end_time`, the current shift overlaps at least one earlier shift.

---

# SQL (portable / Snowflake style)

```sql
WITH ordered_shifts AS (
  SELECT
    employee_id,
    start_time,
    end_time,
    -- running maximum of end_time over all preceding rows for the same employee
    MAX(end_time) OVER (
      PARTITION BY employee_id
      ORDER BY start_time, end_time
      ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ) AS prev_max_end
  FROM EmployeeShifts
)
SELECT
  employee_id,
  SUM(CASE WHEN prev_max_end IS NOT NULL AND start_time < prev_max_end THEN 1 ELSE 0 END) 
    AS overlapping_shifts
FROM ordered_shifts
GROUP BY employee_id
ORDER BY employee_id;
```

Notes:

* We order by `start_time, end_time` to make ordering deterministic when multiple shifts start at the same time.
* We use `ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING` so `prev_max_end` excludes the current row (so we only compare to *earlier* shifts).
* The overlap condition is `start_time < prev_max_end` (strict), matching the usual definition `earlier.end_time > later.start_time`.

---

# Why this is better

* **Correctness**: It catches overlaps with *any earlier* shift — not just the immediate previous — so it solves the main correctness issue in the LEAD-based attempt.
* **Performance**:

  * Sorting is `O(n log n)` per employee (or `O(N log N)` overall, where `N` = total shifts).
  * The window aggregation and final aggregation are linear, so **no `O(n^2)` self-join** cost.
  * Much faster and more scalable than the self-join when employees have many shifts.
* **Simplicity**: Clear logic, easy to reason about and maintain.

---

# Edge cases & variants

* If you need to **count overlapping pairs** (i.e., how many distinct overlapping pairs of shifts exist), you’d use a self-join (or more careful counting) — this query counts each shift at most once (counts number of shifts that overlap some earlier shift).
* If your DB doesn’t support `ROWS BETWEEN ... 1 PRECEDING` with `MAX()` you can alternatively compute a cumulative max including current row and then compare with `GREATEST(end_time, LAG(cumulative_max))` tricks, but most modern DBs that support analytic functions handle the given form.
* If intervals that touch at endpoints (e.g., `end_time = start_time`) should be considered overlapping, change `start_time < prev_max_end` to `start_time <= prev_max_end`.

---

